In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:


# Define the CTC model architecture
def create_ctc_model(input_shape, num_classes):
    model = tf.keras.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Flatten the output for the dense layers
    model.add(layers.Flatten())
    
    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Set hyperparameters and dataset paths
input_shape = (32, 32, 1)  # Input shape of the handwritten images
num_classes = 26  # Number of classes (assuming alphabets)
batch_size = 64
epochs = 10
train_data_path = 'path/to/train_data.npy'  # Path to the preprocessed training data
train_labels_path = 'path/to/train_labels.npy'  # Path to the corresponding training labels

# Load the preprocessed training data and labels
train_data = np.load(train_data_path)
train_labels = np.load(train_labels_path)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2)

# Convert labels to sparse representation
train_labels_sparse = tf.keras.backend.ctc_label_dense_to_sparse(train_labels)
val_labels_sparse = tf.keras.backend.ctc_label_dense_to_sparse(val_labels)

# Create the CTC model
model = create_ctc_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.backend.ctc_batch_cost, metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels_sparse, validation_data=(val_data, val_labels_sparse), batch_size=batch_size, epochs=epochs)

# Save the trained model
model.save('handwritten_text_model.h5')

# Load the trained model for inference
model = tf.keras.models.load_model('handwritten_text_model.h5')

# Perform inference on a test image
def recognize_handwritten_text(image):
    # Preprocess the input image (resize, normalize, etc.)
    preprocessed_image = preprocess(image)
    
    # Expand dimensions to match the model's input shape
    input_image = np.expand_dims(preprocessed_image, axis=0)
    
    # Perform inference
    prediction = model.predict(input_image)
    
    # Decode the predicted labels using CTC decoding algorithm
    decoded_text = tf.keras.backend.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1])[0][0]
    
    # Convert the decoded labels to text
    recognized_text = ''
    for label in decoded_text:
        recognized_text += chr(label+ord('A'))  # Assuming labels are mapped to A-Z
    
    return recognized_text

# Usage example
test_image_path = 'path/to/test_image.jpg'  # Path to the test image
test_image = load_image(test_image_path)
recognized_text = recognize_handwritten_text(test_image)
print('Recognized Text:', recognized_text)
